In [1]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic   # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def count_files(folder_path):
    folder_len = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
    return folder_len

In [7]:
def read_file_names(folder_path):
    file_name = [name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))]
    return file_name

In [9]:
def mediapipe_detection(image, model):    
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB) # Color conversion from BGR to RGB
    image.flags.writeable = False                  # Image is no longe writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR) # Color converson from RGB to BGR
    return image, results

In [11]:
def draw_styled_landmarks(image, results):
    # Draw Face connections - if we want just outlines of the face
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )

                            # OR if we want mesh
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          )
    
    # Draw Pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    
    # Draw Left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(121,44,121), thickness=2, circle_radius=2)
                             )
    # Draw Right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [13]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [17]:
# Path for exported data, numpy arrays  
DATA_PATH = os.path.join('MP_Data')   

# Actions that we try to detect  
actions = np.array(['hello', 'thanks'])  

def count_files(folder_path):  
    # Function to count the number of files in a folder  
    return len(os.listdir(folder_path))  

def read_file_names(folder_path):  
    # Function to read file names in a folder  
    return os.listdir(folder_path)  

with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.5) as holistic:  
    for action in actions:  
        folder_path = "D:/mediapipe_projects/hand_gesture_medium/workspace/images/collected_images/handgestures/{}".format(action)  
        file_names = read_file_names(folder_path)  
        
        for sequence in range(len(file_names)):  
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)), exist_ok=True)  
            for frame_num in range(len(file_names)):  
                file_name = file_names[frame_num]  
                file_path = os.path.join(folder_path, file_name)  
                
                image = cv.imread(file_path)  
                if image is None:  
                    print("Error: could not load the image {}".format(file_name))  
                else:  
                    image, results = mediapipe_detection(image, holistic) # Make Detection  
                    draw_styled_landmarks(image, results) # Draw Landmarks  
                    keypoints = extract_keypoints(results) # Extract key points  
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy")  
                    np.save(npy_path, keypoints)

Error: could not load the image a-young-african-american-male-athlete-is-standing-in-a-stadium-holding-a-phone-and-talking-on-a-video-call-through-headphones-he-looks-at-the-camera-smiles-says-hello-vertical-close-up-photo-2njbrd6.jpg
Error: could not load the image happy-african-american-woman-waving-hello-video-call-conference-with-friends-home-living-room-casual-smiling-student-doing-greeting-hand-gesture-camera-while-talking-with-colleagues_482257-60469.jpg
Error: could not load the image portrait-senior-woman-and-wave-for-video-call-communication-and-social-media-mature-female-hand-gesture-and-greeting-with-smile-hello-or-goodbye-for-connection-recording-or-facetime-for-talking-2PYEM23.jpg
Error: could not load the image pov-of-young-woman-making-online-selfie-video-call-talking-waving-hand-hello-looking-at-camera-at-home-apartment-indoors-communication-meeting-redhead-girl-blogger-making-smartphone-conversation-2R8D122.jpg
Error: could not load the image a-young-african-american-

In [18]:
sequences, labels = [], []
num_files = count_files(folder_path)
actions = np.array(["no"])
label_map = {label:num for num, label in enumerate(actions)}

for action in actions:
    for sequence in range(num_files):
        window = []
        for frame_num in range(num_files):            
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))

            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

FileNotFoundError: [Errno 2] No such file or directory: 'MP_Data_Updated_check\\no\\86\\7.npy'

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.57852125, 0.47353992, 0.8479135 , ..., 0.        , 0.        ,
        0.        ]),
 array([ 0.69369483,  0.40353879, -0.18942356, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.39580855,  0.32564956,  0.25817749, ...,  0.19464728,
         0.34147066, -0.01513922]),
 array([ 0.16030587,  0.15580127, -0.62466204, ...,  0.        ,
         0.        ,  0.        ]),
 array([0.54687846, 0.29298449, 0.06400084, ..., 0.        , 0.        ,
        0.        ]),
 array([0.33074185, 0.47009414, 0.39036959, ..., 0.        , 0.        ,
        0.        ]),
 array([ 0.7151106 ,  0.40413657,  0.34600526, ...,  0.55406725,
         0.397394  , -0.03066501]),
 array([ 0.17014238,  0.68213475, -0.0

In [22]:
np.array(sequences[0]).shape

(220, 1662)

In [ ]:
np.array(labels).shape